# Load Data

In [1]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np
#import pandas as pd

%load_ext autoreload
%autoreload 2
data_train = []
data_test = []

data_train = load_csv_data('Data/train.csv', False)
data_test = load_csv_data('Data/test.csv', False)

oy = data_train[0]
otx = data_train[1]
oids = data_train[2]
otest_y = data_test[0]
otest_tx = data_test[1]
otest_ids = data_test[2]
#oy, otx, oids = load_csv_data('Data/train.csv', False)
#otest_y, otest_tx, otest_ids= load_csv_data('Data/test.csv', False)

#data = pd.read_csv('Data/train.csv', sep=",")


In [2]:
#pd.plotting.scatter_matrix(data)

In [3]:
def prepro(M,c):
    return addFunckyThings(addColumns(M, c), M)

In [ ]:
from preprocessing import *

# Finding the Best columns
best_number_of_colums = np.zeros(4)
best_per_of_columns = np.zeros(4)

for nbColumns in range(8, 10):
#for nbColumns in [2, 5, 10]:
    y, tx, ids = oy, otx, oids
    test_y, test_tx,test_ids = otest_y, otest_tx, otest_ids
    # Preprocessing


    PRI_jet_num_colomn_train = tx[:, 22]
    PRI_jet_num_colomn_test = test_tx[:, 22]


    y_clustered = list()
    tx_clustered = list()
    ids_clustered = list()
    test_y_clustered = list()
    test_tx_clustered = list()
    test_ids_clustered = list()
   

    for i in range(4):
        indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]
        

        y_clustered.append(y[indices])
        tx_clustered.append(tx[indices])
        ids_clustered.append(ids[indices])

        test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]
        
        test_y_clustered.append(test_y[test_indices])
        test_tx_clustered.append(test_tx[test_indices])
        test_ids_clustered.append(test_ids[test_indices])

        #delete colinear columns and preprocess the data
        indices_to_delete = list()
        for col in range(tx_clustered[i].shape[1]):
            if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
                indices_to_delete.append(col)

        deleted_tx = np.delete(tx_clustered[i], indices_to_delete, 1)
        tx_clustered[i] = prepro(deleted_tx, nbColumns)
        
        deleted_tx = np.delete(test_tx_clustered[i], indices_to_delete, 1)
        test_tx_clustered[i] = prepro(deleted_tx, nbColumns)


    # Cross-Validation & Weight computation

    from cross_validation import *
    w = list()
    result = list()
    

    loss = 0
    perGood = 0
    maxW = 0
    for i in range(4):
        wi,loss_te, perGoodI = cross_validation_run(tx_clustered[i], y_clustered[i])
        if(perGoodI > best_per_of_columns[i]):
            best_per_of_columns[i] = perGoodI
            best_number_of_colums[i] = nbColumns
        
        w.append(wi)
        maxW += np.max(np.abs(wi))/4
        loss += loss_te/4
        perGood += perGoodI/4
        test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])
        
    print("Number of columns : ", nbColumns, " loss : ", loss, "Max of w : ", maxW,  " Percentage of true Y : ", perGood)
print(best_number_of_colums) 

best lambda:  5.42867543932e-09 best perc:  0.838177596285
best lambda:  3.23745754282e-10 best perc:  0.83797742013
best lambda:  1.84206996933e-07 best perc:  0.845303867403
best lambda:  4.71486636346e-07 best perc:  0.848706862039
best lambda:  3.72759372031e-07 best perc:  0.794026617146
best lambda:  1.20679264064e-06 best perc:  0.796966883318
best lambda:  6.25055192527e-06 best perc:  0.799442896936
best lambda:  1.67683293681e-09 best perc:  0.793562364593
best lambda:  2.32995181052e-07 best perc:  0.825154835636
best lambda:  1.38949549437e-08 best perc:  0.824678418294
best lambda:  7.19685673001e-08 best perc:  0.825393044307
best lambda:  5.68986602902e-08 best perc:  0.824837224075
best lambda:  4.09491506238e-10 best perc:  0.808157372315
best lambda:  1.26485521686e-10 best perc:  0.821692835228
best lambda:  6.5512855686e-10 best perc:  0.80274318715
best lambda:  1.09854114199e-08 best perc:  0.814293448836
Number of columns :  8  loss :  1.16944732036e+17 Max of w 

best lambda:  4.49843266897e-08 best perc:  0.829890303467
best lambda:  4.71486636346e-07 best perc:  0.82680759068
best lambda:  7.54312006335e-07 best perc:  0.836456081352
best lambda:  6.86648845004e-09 best perc:  0.837416926896
best lambda:  4.49843266897e-08 best perc:  0.772000412669
best lambda:  3.08884359648e-06 best perc:  0.773496337563
best lambda:  2.02358964773e-05 best perc:  0.776591354586
best lambda:  1.20679264064e-06 best perc:  0.772877334159
best lambda:  1.93069772888e-06 best perc:  0.79942829919
best lambda:  3.23745754282e-05 best perc:  0.798316658726
best lambda:  8.28642772855e-05 best perc:  0.798078450056
best lambda:  1.20679264064e-06 best perc:  0.798872478958
best lambda:  4.94171336132e-06 best perc:  0.771521386031
best lambda:  0.000212095088792 best perc:  0.792997653853
best lambda:  1.20679264064e-06 best perc:  0.76863382061
best lambda:  3.23745754282e-05 best perc:  0.778920772424
Number of columns :  2  loss :  2.14732779545 Max of w :  324628.78522  Percentage of true Y :  0.795769116326
best lambda:  2.02358964773e-05 best perc:  0.829810233005
best lambda:  2.02358964773e-05 best perc:  0.826847625911
best lambda:  5.17947467923e-05 best perc:  0.836215869966
best lambda:  2.02358964773e-05 best perc:  0.837136680279
best lambda:  5.17947467923e-05 best perc:  0.771845661818
best lambda:  4.94171336132e-06 best perc:  0.773290003095
best lambda:  2.02358964773e-05 best perc:  0.776642938203
best lambda:  2.02358964773e-05 best perc:  0.772877334159
best lambda:  4.94171336132e-06 best perc:  0.79950770208
best lambda:  5.17947467923e-05 best perc:  0.798316658726
best lambda:  8.28642772855e-05 best perc:  0.798237255836
best lambda:  7.90604321091e-06 best perc:  0.798872478958
best lambda:  8.28642772855e-05 best perc:  0.77170185887
best lambda:  0.000212095088792 best perc:  0.792997653853
best lambda:  7.90604321091e-06 best perc:  0.768453347771
best lambda:  5.17947467923e-05 best perc:  0.778920772424
Number of columns :  3  loss :  2.1029037926 Max of w :  2.60080969768  Percentage of true Y :  0.795729629685
best lambda:  2.81176869797e-08 best perc:  0.83081111378
best lambda:  4.94171336132e-06 best perc:  0.829690127312
best lambda:  1.20679264064e-06 best perc:  0.839018336136
best lambda:  6.86648845004e-09 best perc:  0.840179357835
best lambda:  7.54312006335e-07 best perc:  0.77669452182
best lambda:  3.08884359648e-06 best perc:  0.776797689054
best lambda:  4.71486636346e-07 best perc:  0.78262663778
best lambda:  2.94705170255e-07 best perc:  0.776127102032
best lambda:  1.93069772888e-06 best perc:  0.806892170875
best lambda:  3.23745754282e-05 best perc:  0.805621724631
best lambda:  1.20679264064e-06 best perc:  0.80530411307
best lambda:  1.20679264064e-06 best perc:  0.806733365094
best lambda:  4.94171336132e-06 best perc:  0.783793539072
best lambda:  3.08884359648e-06 best perc:  0.804728388378
best lambda:  7.90604321091e-06 best perc:  0.782349756362
best lambda:  3.08884359648e-06 best perc:  0.795885219275
Number of columns :  4  loss :  8.18545189593 Max of w :  5065.69012475  Percentage of true Y :  0.802703322657
best lambda:  2.94705170255e-07 best perc:  0.833733685643
best lambda:  1.84206996933e-07 best perc:  0.832332452558
best lambda:  0.000212095088792 best perc:  0.840419569221
best lambda:  4.49843266897e-08 best perc:  0.844222916166
best lambda:  0.000132571136559 best perc:  0.785205818632
best lambda:  4.49843266897e-08 best perc:  0.785927989271
best lambda:  1.26485521686e-05 best perc:  0.790570514805
best lambda:  1.15139539933e-07 best perc:  0.784277313525
best lambda:  2.02358964773e-05 best perc:  0.812767984755
best lambda:  1.26485521686e-05 best perc:  0.811179926949
best lambda:  0.0596362331659 best perc:  0.811021121169
best lambda:  1.93069772888e-06 best perc:  0.813403207877
best lambda:  1.26485521686e-05 best perc:  0.786861577333
best lambda:  4.94171336132e-06 best perc:  0.810683992059
best lambda:  1.93069772888e-06 best perc:  0.789027251399
best lambda:  4.94171336132e-06 best perc:  0.796968056308
Number of columns :  5  loss :  6153009.90771 Max of w :  12.5800901777  Percentage of true Y :  0.808037711104
best lambda:  2.81176869797e-08 best perc:  0.836015693811
best lambda:  3.23745754282e-05 best perc:  0.835094883497
best lambda:  4.49843266897e-08 best perc:  0.844583233245
best lambda:  2.02358964773e-05 best perc:  0.846665065257
best lambda:  7.54312006335e-07 best perc:  0.790880016507
best lambda:  3.23745754282e-05 best perc:  0.793201279274
best lambda:  2.02358964773e-05 best perc:  0.796812132467
best lambda:  1.84206996933e-07 best perc:  0.79134426906
best lambda:  1.93069772888e-06 best perc:  0.819834841988
best lambda:  4.71486636346e-07 best perc:  0.818246784183
best lambda:  0.0232995181052 best perc:  0.821343496903
best lambda:  3.08884359648e-06 best perc:  0.819596633317
best lambda:  3.08884359648e-06 best perc:  0.80129940444
best lambda:  5.17947467923e-05 best perc:  0.81988810684
best lambda:  3.23745754282e-05 best perc:  0.795163327919
best lambda:  0.00222299648253 best perc:  0.810142573543
Number of columns :  6  loss :  130445959683.0 Max of w :  23690.9117144  Percentage of true Y :  0.815006983891
best lambda:  2.81176869797e-08 best perc:  0.836576187045
best lambda:  1.84206996933e-07 best perc:  0.836736327969
best lambda:  4.94171336132e-06 best perc:  0.844743374169
best lambda:  0.000132571136559 best perc:  0.847786051725
best lambda:  1.26485521686e-05 best perc:  0.793768699061
best lambda:  7.54312006335e-07 best perc:  0.796966883318
best lambda:  1.26485521686e-05 best perc:  0.800268234809
best lambda:  1.84206996933e-07 best perc:  0.793768699061
best lambda:  1.93069772888e-06 best perc:  0.823804986502
best lambda:  1.93069772888e-06 best perc:  0.824599015404
best lambda:  3.23745754282e-05 best perc:  0.824281403843
best lambda:  1.20679264064e-06 best perc:  0.824837224075
best lambda:  0.000132571136559 best perc:  0.808698790832
best lambda:  0.000542867543932 best perc:  0.823858509294
best lambda:  0.00568986602902 best perc:  0.800397040245
best lambda:  1.93069772888e-06 best perc:  0.81519581303
Number of columns :  7  loss :  1.09092094619e+14 Max of w :  21690.1703838  Percentage of true Y :  0.818517952524
best lambda:  4.49843266897e-08 best perc:  0.838057490592
best lambda:  1.15139539933e-07 best perc:  0.837937384899
best lambda:  7.54312006335e-07 best perc:  0.845303867403
best lambda:  7.19685673001e-08 best perc:  0.848666826808
best lambda:  1.20679264064e-06 best perc:  0.793820282678
best lambda:  7.90604321091e-06 best perc:  0.797018466935
best lambda:  1.93069772888e-06 best perc:  0.799339729702
best lambda:  1.15139539933e-07 best perc:  0.793459197359
best lambda:  1.20679264064e-06 best perc:  0.824202000953
best lambda:  1.26485521686e-05 best perc:  0.824440209624
best lambda:  7.90604321091e-06 best perc:  0.824916626965
best lambda:  0.0372759372031 best perc:  0.824043195172
best lambda:  3.08884359648e-06 best perc:  0.80905973651
best lambda:  0.00568986602902 best perc:  0.821331889551
best lambda:  1.93069772888e-06 best perc:  0.800216567407
best lambda:  3.23745754282e-05 best perc:  0.814112975997
Number of columns :  8  loss :  1.63100252138e+16 Max of w :  416411.62474  Percentage of true Y :  0.818495403035
best lambda:  7.54312006335e-07 best perc:  0.839538794139
best lambda:  1.26485521686e-05 best perc:  0.8407798863
best lambda:  1.84206996933e-07 best perc:  0.846464889102
best lambda:  7.19685673001e-08 best perc:  0.849747778045
best lambda:  4.71486636346e-07 best perc:  0.798308057361
best lambda:  0.00222299648253 best perc:  0.804033838853
best lambda:  0.00910298177992 best perc:  0.80496234396
best lambda:  2.94705170255e-07 best perc:  0.799803982255
best lambda:  1.26485521686e-05 best perc:  0.826663490551
best lambda:  7.54312006335e-07 best perc:  0.827298713673
best lambda:  0.0372759372031 best perc:  0.82944259171
best lambda:  0.014563484775 best perc:  0.831348261077
best lambda:  0.625055192527 best perc:  0.807976899477
best lambda:  0.0372759372031 best perc:  0.826385129038
best lambda:  2.02358964773e-05 best perc:  0.8043674427
best lambda:  7.90604321091e-06 best perc:  0.821873308067
Number of columns :  9  loss :  4.68496096021e+19 Max of w :  60603.5257595  Percentage of true Y :  0.822437212894
best lambda:  2.94705170255e-07 best perc:  0.840659780607
best lambda:  4.71486636346e-07 best perc:  0.841700696613
best lambda:  1.93069772888e-06 best perc:  0.846945311874
best lambda:  4.71486636346e-07 best perc:  0.85154936344
best lambda:  1.84206996933e-07 best perc:  0.803002166512
best lambda:  2.94705170255e-07 best perc:  0.811616630558
best lambda:  1.84206996933e-07 best perc:  0.811358712473
best lambda:  3.08884359648e-06 best perc:  0.804240173321
best lambda:  1.93069772888e-06 best perc:  0.834444973797
best lambda:  5.17947467923e-05 best perc:  0.835636017151
best lambda:  0.00910298177992 best perc:  0.838812132762
best lambda:  0.095409547635 best perc:  0.835636017151
best lambda:  0.0596362331659 best perc:  0.818444324129
best lambda:  1.93069772888e-06 best perc:  0.832701678397
best lambda:  0.625055192527 best perc:  0.812669193286
best lambda:  0.014563484775 best perc:  0.831077422848
Number of columns :  10  loss :  5.4531573258e+26 Max of w :  10.4621954724  Percentage of true Y :  0.828155912182
best lambda:  1.84206996933e-07 best perc:  0.840019216911
best lambda:  2.02358964773e-05 best perc:  0.842101048923
best lambda:  4.71486636346e-07 best perc:  0.84678517095
best lambda:  1.84206996933e-07 best perc:  0.851108975899
best lambda:  1.26485521686e-05 best perc:  0.805581347364
best lambda:  3.08884359648e-06 best perc:  0.812184050346
best lambda:  0.000868511373751 best perc:  0.812441968431
best lambda:  7.54312006335e-07 best perc:  0.807335190344
best lambda:  3.08884359648e-06 best perc:  0.838097506749
best lambda:  0.00910298177992 best perc:  0.837859298078
best lambda:  4.94171336132e-06 best perc:  0.842226457043
best lambda:  0.0372759372031 best perc:  0.837859298078
best lambda:  3.08884359648e-06 best perc:  0.825121819166
best lambda:  0.000542867543932 best perc:  0.841183901823
best lambda:  1.26485521686e-05 best perc:  0.81988810684
best lambda:  4.94171336132e-06 best perc:  0.837754917885
Number of columns :  11  loss :  1.18922801422e+31 Max of w :  103.567246895  Percentage of true Y :  0.831096767177
best lambda:  3.08884359648e-06 best perc:  0.840419569221
best lambda:  0.000542867543932 best perc:  0.650812715189
best lambda:  1.0 best perc:  0.647930178557
best lambda:  5.17947467923e-05 best perc:  0.851509328209
best lambda:  2.02358964773e-05 best perc:  0.805426596513
best lambda:  0.0232995181052 best perc:  0.807541524812
best lambda:  0.00568986602902 best perc:  0.812029299494
best lambda:  1.20679264064e-06 best perc:  0.806303518003
best lambda:  1.93069772888e-06 best perc:  0.837859298078
best lambda:  3.08884359648e-06 best perc:  0.838812132762
best lambda:  0.000542867543932 best perc:  0.842623471494
best lambda:  0.00033932217719 best perc:  0.837859298078
best lambda:  3.23745754282e-05 best perc:  0.82602418336
best lambda:  0.00138949549437 best perc:  0.841905793178
best lambda:  0.00355648030622 best perc:  0.818805269807
best lambda:  0.014563484775 best perc:  0.837754917885
Number of columns :  12  loss :  6.1688619834e+36 Max of w :  60578083618.5  Percentage of true Y :  0.806476068415
best lambda:  2.5595479227e-10 best perc:  0.596044519177
best lambda:  6.5512855686e-10 best perc:  0.654696132597
best lambda:  1.59985871961e-10 best perc:  0.583993914645
best lambda:  4.94171336132e-06 best perc:  0.845423973096
best lambda:  2.02358964773e-05 best perc:  0.805684514598
best lambda:  0.014563484775 best perc:  0.807283606727
best lambda:  0.095409547635 best perc:  0.810791292685
best lambda:  7.90604321091e-06 best perc:  0.805581347364
best lambda:  0.000212095088792 best perc:  0.838494521201
best lambda:  0.000542867543932 best perc:  0.837144672066
best lambda:  4.94171336132e-06 best perc:  0.815150071463
best lambda:  0.244205309455 best perc:  0.817770366841
best lambda:  0.00355648030622 best perc:  0.82764843891
best lambda:  7.90604321091e-06 best perc:  0.82764843891
best lambda:  0.000868511373751 best perc:  0.818444324129
best lambda:  0.00222299648253 best perc:  0.775311315647
Number of columns :  13  loss :  5.43802655581e+40 Max of w :  2.40017767585e+19  Percentage of true Y :  0.779194465628
best lambda:  7.54312006335e-07 best perc:  0.580751060934
best lambda:  4.29193426013e-09 best perc:  0.687645127712
best lambda:  2.5595479227e-10 best perc:  0.62967411322
best lambda:  7.54312006335e-07 best perc:  0.845944431099
best lambda:  0.244205309455 best perc:  0.803105333746
best lambda:  1.93069772888e-06 best perc:  0.809295367791
best lambda:  0.0232995181052 best perc:  0.8105333746
best lambda:  3.08884359648e-06 best perc:  0.806509852471
best lambda:  5.17947467923e-05 best perc:  0.834365570907
best lambda:  8.28642772855e-05 best perc:  0.833015721772
best lambda:  0.00033932217719 best perc:  0.838097506749
best lambda:  0.0372759372031 best perc:  0.835000794029
best lambda:  8.28642772855e-05 best perc:  0.824941346327
best lambda:  0.000132571136559 best perc:  0.836672080852
best lambda:  3.23745754282e-05 best perc:  0.813932503158
best lambda:  1.0 best perc:  0.822775672261
Number of columns :  14  loss :  4.56945528404e+46 Max of w :  1.14320152013e+20  Percentage of true Y :  0.788266241102
[ 10.  11.  12.  12.]

In [21]:
print(best_per_of_columns)
print(best_number_of_colums) 

[ 0.84521379  0.80938564  0.83928855  0.83112254]
[ 10.  11.  12.  12.]


# Using least square on [cst, tx, tx^2, ...] 
31 - 0.695085425438 

61 - 0.776

91 - 0.787

121 - 0.796

151 - 0.805

181 - 0.807

211 - 0.809

241 - 0.816

271 - 0.822

# Computing with good columns

In [22]:
y, tx, ids = oy, otx, oids
test_y, test_tx,test_ids = otest_y, otest_tx, otest_ids
# Preprocessing


PRI_jet_num_colomn_train = tx[:, 22]
PRI_jet_num_colomn_test = test_tx[:, 22]


y_clustered = list()
tx_clustered = list()
ids_clustered = list()
test_y_clustered = list()
test_tx_clustered = list()
test_ids_clustered = list()


for i in range(4):
    indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]
 

    y_clustered.append(y[indices])
    tx_clustered.append(tx[indices])
    ids_clustered.append(ids[indices])

    test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]

    test_y_clustered.append(test_y[test_indices])
    test_tx_clustered.append(test_tx[test_indices])
    test_ids_clustered.append(test_ids[test_indices])

    #delete colinear columns and preprocess the data
    indices_to_delete = list()
    for col in range(tx_clustered[i].shape[1]):
        if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
            indices_to_delete.append(col)

    deleted_tx = np.delete(tx_clustered[i], indices_to_delete, 1)
    tx_clustered[i] = prepro(deleted_tx, int(best_number_of_colums[i]))#addColumns(deleted_tx, int(best_number_of_colums[i]))

    deleted_tx = np.delete(test_tx_clustered[i], indices_to_delete, 1)
    test_tx_clustered[i] = prepro(deleted_tx, int(best_number_of_colums[i]))#addColumns(deleted_tx, int(best_number_of_colums[i]))


# Cross-Validation & Weight computation

from cross_validation import *
w = list()
result = list()


loss = 0
perGood = 0
maxW = 0
for i in range(4):
    wi,loss_te, perGoodI = cross_validation_run(tx_clustered[i], y_clustered[i])
    if(perGoodI > best_per_of_columns[i]):
        best_per_of_columns[i] = perGoodI
        best_number_of_colums[i] = nbColumns

    w.append(wi)
    maxW += np.max(np.abs(wi))/4
    loss += loss_te/4
    perGood += perGoodI/4
    test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])

print(" loss : ", loss, "Max of w : ", maxW,  " Percentage of true Y : ", perGood)

best lambda:  2.94705170255e-07 best perc:  0.840659780607
best lambda:  4.71486636346e-07 best perc:  0.841700696613
best lambda:  1.93069772888e-06 best perc:  0.846945311874
best lambda:  4.71486636346e-07 best perc:  0.85154936344
best lambda:  1.26485521686e-05 best perc:  0.805581347364
best lambda:  3.08884359648e-06 best perc:  0.812184050346
best lambda:  0.000868511373751 best perc:  0.812441968431
best lambda:  7.54312006335e-07 best perc:  0.807335190344
best lambda:  1.93069772888e-06 best perc:  0.837859298078
best lambda:  3.08884359648e-06 best perc:  0.838812132762
best lambda:  0.000542867543932 best perc:  0.842623471494
best lambda:  0.00033932217719 best perc:  0.837859298078
best lambda:  3.23745754282e-05 best perc:  0.82602418336
best lambda:  0.00138949549437 best perc:  0.841905793178
best lambda:  0.00355648030622 best perc:  0.818805269807
best lambda:  0.014563484775 best perc:  0.837754917885
 loss :  5.4531573258e+26 Max of w :  11.0549588135  Percentage 

best lambda:  2.94705170255e-07 best perc:  0.840659780607
best lambda:  4.71486636346e-07 best perc:  0.841700696613
best lambda:  1.93069772888e-06 best perc:  0.846945311874
best lambda:  4.71486636346e-07 best perc:  0.85154936344
best lambda:  1.26485521686e-05 best perc:  0.805581347364
best lambda:  3.08884359648e-06 best perc:  0.812184050346
best lambda:  0.000868511373751 best perc:  0.812441968431
best lambda:  7.54312006335e-07 best perc:  0.807335190344
best lambda:  1.93069772888e-06 best perc:  0.837859298078
best lambda:  3.08884359648e-06 best perc:  0.838812132762
best lambda:  0.000542867543932 best perc:  0.842623471494
best lambda:  0.00033932217719 best perc:  0.837859298078
best lambda:  3.23745754282e-05 best perc:  0.82602418336
best lambda:  0.00138949549437 best perc:  0.841905793178
best lambda:  0.00355648030622 best perc:  0.818805269807
best lambda:  0.014563484775 best perc:  0.837754917885
 loss :  5.4531573258e+26 Max of w :  11.0549588135  Percentage of true Y :  0.831252629604

old best: 0.826606619791

new best: 0.827178252354

intermediate best: 0.830793909724 (np.logspace(-5, 0, 15))

new new best (lambdas): 0.831252629604



# Analysing why it isn't working

In [ ]:

for i in range(4):
    realPred = np.dot(tx_clustered[i], w[i])
    #print(np.max(realPred))
    #df = pd.DataFrame(realPred)
    #df.hist(bins=300)
    y_pred = predict_labels(w[i],tx_clustered[i])
    
    false_index = [ind for ind,a in enumerate(y_clustered[i]) if y_pred[ind] != a]
    
    print(np.mean(realPred[false_index]))
    print("Per : ", percentageGood(y_clustered[i],tx_clustered[i],w[i]), "Len : ", len(y_clustered[i]) ) 
    
    print(len(w[i]))


# Post Processing

In [24]:
test_ids = [item for sublist in test_ids_clustered for item in sublist]
y_pred = [item for sublist in test_y_clustered for item in sublist]

test_ids, y_pred = zip(*sorted(zip(test_ids, y_pred)))

# Output Data

In [25]:
name = 'little_lambda_submission.csv'
create_csv_submission(test_ids, y_pred, name)

In [26]:
np.zeros(4)

array([ 0.,  0.,  0.,  0.])

In [ ]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
from preprocessing import *
from experimentation import *
import numpy as np

%load_ext autoreload
%autoreload 2
print("1")
#Data Loading
data_train = []
data_test = []
print("2")
data_train = load_csv_data('Data/train.csv', False)
data_test = load_csv_data('Data/test.csv', False)
print("3")
print("--- Data Loaded ---")

best_perf_of_columns = [0, 0, 0, 0] #np.full((4), 0)
min_poly_degree = 8
max_poly_degree = 13

y_clustered, tx_clustered, ids_clustered, test_y_clustered, test_tx_clustered, test_ids_clustered = prepare_clusters(data_train, data_test)
print("4")
for i in range(min_poly_degree, max_poly_degree+1):
    coeffArr = [i, i, i, i] #np.full((4), i)
    tx_clustered_, test_tx_clustered_ = preprocessing(tx_clustered, test_tx_clustered, coeffArr)
    w, perf_of_columns, predictions = search_best_polynomial_fit(i, y_clustered, tx_clustered_, test_y_clustered, test_tx_clustered_)
    for index, el in enumerate(perf_of_columns):
        if el > best_perf_of_columns[index]:
            best_perf_of_columns[index] = el
            test_y_clustered[index] = predictions[index]

# Post Processing
test_ids = [item for sublist in test_ids_clustered for item in sublist]
y_pred = [item for sublist in test_y_clustered for item in sublist]

test_ids, y_pred = zip(*sorted(zip(test_ids, y_pred)))

# Output Data
name = 'submission.csv'
create_csv_submission(test_ids, y_pred, name)



1
2
3
--- Data Loaded ---
4
best lambda:  5.42867543932e-09 best perc:  0.838177596285
best lambda:  3.23745754282e-10 best perc:  0.83797742013
best lambda:  1.84206996933e-07 best perc:  0.845303867403
best lambda:  4.71486636346e-07 best perc:  0.848706862039
best lambda:  5.68986602902e-08 best perc:  0.7808727948
best lambda:  5.17947467923e-10 best perc:  0.782110801609
best lambda:  6.25055192527e-06 best perc:  0.785927989271
best lambda:  3.90693993705e-06 best perc:  0.781801299907
best lambda:  1e-10 best perc:  0.791885024615
best lambda:  4.94171336132e-06 best perc:  0.796093377799
best lambda:  1.52641796718e-06 best perc:  0.802922026362
best lambda:  6.25055192527e-06 best perc:  0.801969191679
best lambda:  7.90604321091e-06 best perc:  0.796968056308
best lambda:  1.93069772888e-06 best perc:  0.818083378452
best lambda:  1.20679264064e-06 best perc:  0.797329001985
best lambda:  9.10298177992e-08 best perc:  0.815556758708
Number of columns :  8  loss :  1.169447320